Teacher student model, Teacher GRU, Student MLP

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from bayes_opt import BayesianOptimization
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1
    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
    return X_scaled, y, scaler

# ============================================
# 3. Build GRU Model (Teacher)
# ============================================
def build_gru_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(num_units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Knowledge Distillation: Student Model
# ============================================
def build_student_model(input_shape, num_classes=4, num_units=32):
    model = Sequential()
    model.add(Dense(num_units, input_dim=input_shape[0], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_with_distillation(teacher_model, X_train, y_train, student_model):
    teacher_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    soft_targets = teacher_model.predict(X_train)
    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    start_train = time.time()
    student_model.fit(X_train_flattened, soft_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    training_time = time.time() - start_train

    start_infer = time.time()
    _ = student_model.predict(X_train_flattened[:10])  # warmup
    inference_time = (time.time() - start_infer) / 10.0
    testing_time = time.time() - start_infer

    student_model.save("temp_student_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_student_model.h5") / (1024 ** 2)
    os.remove("temp_student_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in student_model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Student Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return student_model

# ============================================
# 7. Train & Evaluate Teacher Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Teacher Model Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 8. Main Function
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final GRU Model (Teacher)...")
    teacher_model, _ = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

    input_shape_flat = (X_train.shape[1] * X_train.shape[2],)
    student_model = build_student_model(input_shape=input_shape_flat)

    print("\nTraining Student Model using Knowledge Distillation...")
    student_model = train_with_distillation(teacher_model, X_train, y_train, student_model)

    print("\nEvaluating Student Model...")
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)
    y_pred_student = student_model.predict(X_test_flattened).argmax(axis=1)
    acc_student = accuracy_score(y_test, y_pred_student)
    cm_student = confusion_matrix(y_test, y_pred_student)

    print("\n===== Student Model Evaluation =====")
    print(f"Accuracy: {acc_student:.4f}")
    print("Report:\n", classification_report(y_test, y_pred_student, digits=4))
    print("Confusion Matrix:\n", cm_student)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9959    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9959    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 62.98     | 0.2457    | 98.0      | 123.8     |
| 9         | 1.0       | 33.36     | 0.4432    | 99.96     | 127.6     |
| 10        | 1.0       | 45.93     | 0.4456    | 99.59     | 127.7     |
| 11        | 1.0       | 18.27     | 0.3674    | 99.38     | 128.0     |
| 12   


===== Teacher Model Evaluation =====
Accuracy: 0.9803
Report:
               precision    recall  f1-score   support

           0     0.8611    0.7381    0.7949        42
           1     0.9526    0.9779    0.9651       226
           2     1.0000    1.0000    1.0000       509
           3     1.0000    1.0000    1.0000        36

    accuracy                         0.9803       813
   macro avg     0.9534    0.9290    0.9400       813
weighted avg     0.9796    0.9803    0.9797       813

Confusion Matrix:
 [[ 31  11   0   0]
 [  5 221   0   0]
 [  0   0 509   0]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 27.17s
Inference/sample: 0.001200s
Testing Time: 0.98s
Model Size: 0.25 MB
Trainable Params: 60,136
RAM Usage: 1225.85 MB

Training Student Model using Knowledge Distillation...
Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9955 - loss: 0.0205 - val_accuracy: 0.9959 - val_loss: 0.0126
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accura


===== Student Model Metrics =====
Training Time: 2.98s
Inference/sample: 0.011994s
Testing Time: 0.12s
Model Size: 0.02 MB
Trainable Params: 388
RAM Usage: 1234.47 MB

Evaluating Student Model...
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

===== Student Model Evaluation =====
Accuracy: 0.8844
Report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        42
           1     0.7473    0.9027    0.8176       226
           2     0.9543    0.9430    0.9486       509
           3     0.9459    0.9722    0.9589        36

    accuracy                         0.8844       813
   macro avg     0.6619    0.7045    0.6813       813
weighted avg     0.8471    0.8844    0.8637       813

Confusion Matrix:
 [[  0  42   0   0]
 [  0 204  22   0]
 [  0  27 480   2]
 [  0   0   1  35]]


: 

In [1]:
import pandas as pd
import numpy as np
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1
    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # Keep 3D shape for GRU
    return X_scaled, y, scaler

# ============================================
# 3. Build GRU Model (Teacher)
# ============================================
def build_gru_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(num_units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Knowledge Distillation: Student Model
# ============================================
def build_student_model(input_shape, num_classes=4, num_units=32):
    model = Sequential()
    model.add(GRU(num_units, input_shape=input_shape))  # Using GRU for student model as well
    model.add(Dropout(0.2))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_with_distillation(teacher_model, X_train, y_train, student_model):
    teacher_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    soft_targets = teacher_model.predict(X_train)
    
    # Flattening the input for the student model to match 3D input expected by GRU
    X_train_flattened = X_train.reshape(X_train.shape[0], 1, X_train.shape[2])  # Keep 3D shape

    start_train = time.time()
    student_model.fit(X_train_flattened, soft_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    training_time = time.time() - start_train

    start_infer = time.time()
    _ = student_model.predict(X_train_flattened[:10])  # warmup
    inference_time = (time.time() - start_infer) / 10.0
    testing_time = time.time() - start_infer

    student_model.save("temp_student_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_student_model.h5") / (1024 ** 2)
    os.remove("temp_student_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in student_model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Student Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return student_model

# ============================================
# 7. Train & Evaluate Teacher Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Teacher Model Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 8. Main Function
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final GRU Model (Teacher)...")
    teacher_model, _ = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

    input_shape_flat = (X_train.shape[1], X_train.shape[2])  # Keep the 3D shape for GRU
    student_model = build_student_model(input_shape=input_shape_flat)

    print("\nTraining Student Model using Knowledge Distillation...")
    student_model = train_with_distillation(teacher_model, X_train, y_train, student_model)

    print("\nEvaluating Student Model...")
    X_test_flattened = X_test.reshape(X_test.shape[0], 1, X_test.shape[2])  # Keep the 3D shape for GRU
    y_pred_student = student_model.predict(X_test_flattened).argmax(axis=1)
    acc_student = accuracy_score(y_test, y_pred_student)
    cm_student = confusion_matrix(y_test, y_pred_student)

    print("\n===== Student Model Evaluation =====")
    print(f"Accuracy: {acc_student:.4f}")
    print("Report:\n", classification_report(y_test, y_pred_student, digits=4))
    print("Confusion Matrix:\n", cm_student)
 
if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9959    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9959    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 62.98     | 0.2457    | 98.0      | 123.8     |
| 9         | 1.0       | 33.36     | 0.4432    | 99.96     | 127.6     |
| 10        | 1.0       | 45.93     | 0.4456    | 99.59     | 127.7     |
| 11        | 1.0       | 18.27     | 0.3674    | 99.38     | 128.0     |
| 12   


===== Teacher Model Evaluation =====
Accuracy: 0.9840
Report:
               precision    recall  f1-score   support

           0     0.9394    0.7381    0.8267        42
           1     0.9532    0.9912    0.9718       226
           2     1.0000    1.0000    1.0000       509
           3     1.0000    1.0000    1.0000        36

    accuracy                         0.9840       813
   macro avg     0.9731    0.9323    0.9496       813
weighted avg     0.9839    0.9840    0.9832       813

Confusion Matrix:
 [[ 31  11   0   0]
 [  2 224   0   0]
 [  0   0 509   0]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 37.85s
Inference/sample: 0.001952s
Testing Time: 1.59s
Model Size: 0.25 MB
Trainable Params: 60,136
RAM Usage: 1049.91 MB

Training Student Model using Knowledge Distillation...
Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9995 - loss: 0.0154 - val_accuracy: 0.9959 - val_loss: 0.0086
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accura


===== Student Model Metrics =====
Training Time: 12.33s
Inference/sample: 0.075973s
Testing Time: 0.76s
Model Size: 0.04 MB
Trainable Params: 4,532
RAM Usage: 1088.19 MB

Evaluating Student Model...
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

===== Student Model Evaluation =====
Accuracy: 0.7601
Report:
               precision    recall  f1-score   support

           0     0.7500    0.2857    0.4138        42
           1     0.7934    0.9513    0.8652       226
           2     0.9807    0.6974    0.8152       509
           3     0.2195    1.0000    0.3600        36

    accuracy                         0.7601       813
   macro avg     0.6859    0.7336    0.6135       813
weighted avg     0.8830    0.7601    0.7882       813

Confusion Matrix:
 [[ 12  30   0   0]
 [  4 215   7   0]
 [  0  26 355 128]
 [  0   0   0  36]]


Teacher studnet model, Teacher GRU, Student GRU

In [1]:
import pandas as pd
import numpy as np
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from bayes_opt import BayesianOptimization
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1
    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
    return X_scaled, y, scaler

# ============================================
# 3. Build GRU Model (Teacher & Student)
# ============================================
def build_gru_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(num_units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_student_model(input_shape, num_classes=4, num_units=32):
    model = Sequential()
    model.add(GRU(num_units, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=2, n_iter=3)
    return optimizer.max['params']

# ============================================
# 6. Knowledge Distillation: Train Student
# ============================================
def train_with_distillation(teacher_model, X_train, y_train, student_model):
    teacher_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    soft_targets = teacher_model.predict(X_train)

    # Student is also a GRU model — accepts same input shape
    start_train = time.time()
    student_model.fit(X_train, soft_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    training_time = time.time() - start_train

    start_infer = time.time()
    _ = student_model.predict(X_train[:10])
    inference_time = (time.time() - start_infer) / 10.0
    testing_time = time.time() - start_infer

    student_model.save("temp_student_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_student_model.h5") / (1024 ** 2)
    os.remove("temp_student_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in student_model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Student Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return student_model

# ============================================
# 7. Train & Evaluate Teacher Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_gru_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Teacher Model Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 8. Main Function
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final GRU Model (Teacher)...")
    teacher_model, _ = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

    print("\nTraining Student Model using Knowledge Distillation...")
    student_model = build_student_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    student_model = train_with_distillation(teacher_model, X_train, y_train, student_model)

    print("\nEvaluating Student Model...")
    y_pred_student = student_model.predict(X_test).argmax(axis=1)
    acc_student = accuracy_score(y_test, y_pred_student)
    cm_student = confusion_matrix(y_test, y_pred_student)

    print("\n===== Student Model Evaluation =====")
    print(f"Accuracy: {acc_student:.4f}")
    print("Report:\n", classification_report(y_test, y_pred_student, digits=4))
    print("Confusion Matrix:\n", cm_student)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9878    | 63.56     | 0.219     | 54.21     | 32.49     |
| 4         | 1.0       | 17.5      | 0.2856    | 99.41     | 125.8     |
| 5         | 0.9959    | 25.57     | 0.4356    | 53.0      | 114.3     |

Best Params: {'batch_size': 17.502535329294783, 'dropout_rate': 0.2855509267081279, 'epochs': 99.405432700386, 'num_units': 125.81175651469712}

Training Final GRU Model (Teacher)...
Epoch 1/99
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.4861 - loss: 1.2554 - val_accuracy: 0.8008 - val_loss: 0.7676
Epoch 2/99
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7779 - loss: 0.7270 - val_accuracy: 0.8862 - val_loss: 0.4544
Epoch 3/99
58/


===== Teacher Model Evaluation =====
Accuracy: 0.9865
Report:
               precision    recall  f1-score   support

           0     1.0000    0.7381    0.8493        42
           1     0.9536    1.0000    0.9762       226
           2     1.0000    1.0000    1.0000       509
           3     1.0000    1.0000    1.0000        36

    accuracy                         0.9865       813
   macro avg     0.9884    0.9345    0.9564       813
weighted avg     0.9871    0.9865    0.9856       813

Confusion Matrix:
 [[ 31  11   0   0]
 [  0 226   0   0]
 [  0   0 509   0]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 37.07s
Inference/sample: 0.000941s
Testing Time: 0.77s
Model Size: 0.24 MB
Trainable Params: 58,314
RAM Usage: 644.97 MB

Training Student Model using Knowledge Distillation...
Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9995 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy


===== Student Model Metrics =====
Training Time: 6.76s
Inference/sample: 0.044893s
Testing Time: 0.45s
Model Size: 0.04 MB
Trainable Params: 4,532
RAM Usage: 696.77 MB

Evaluating Student Model...
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

===== Student Model Evaluation =====
Accuracy: 0.7171
Report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        42
           1     0.7631    0.9690    0.8538       226
           2     0.9791    0.6444    0.7773       509
           3     0.1885    1.0000    0.3172        36

    accuracy                         0.7171       813
   macro avg     0.4827    0.6534    0.4871       813
weighted avg     0.8335    0.7171    0.7380       813

Confusion Matrix:
 [[  0  42   0   0]
 [  0 219   7   0]
 [  0  26 328 155]
 [  0   0   0  36]]
